In [ ]:
#Only use if sandbox is not installed
import os,sys
sys.path.append('../../../')

## Before starting be sure you have some models in the checkpoints folder 

In [ ]:
from sandbox import _test_data
if len(os.listdir(_test_data["landscape_generation"] + "checkpoints")) == 0:
    from warnings import warn 
    warn('No models to run! You need to download the files using Pooch' 
         '(Check README file "Downloading sample datasets")')
    
    from sandbox.utils.download_sample_datasets import (landscape_models, 
                                                        download_landscape_name)
    
    while True:
        if input("Do you want to download an specific Trained model? [y/n]") == "y":
            print("Available models: %s" % landscape_models)
            model = input("Name of model to download:")
            download_landscape_name(model)
        else:
            break
else:
    print("Available models to run: \n{}".format(os.listdir(_test_data["landscape_generation"] + "checkpoints")))

## Initialize Sensor and Projector

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

In [ ]:
#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="dummy")

In [ ]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)


## Initialize marker detection

In [ ]:
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor = sensor)

## Initialize main thread for live update

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)


In [ ]:
main.run()





### Control the features of the main thread

In [ ]:
main.widget_plot_module()

## Set the path of: pytorch-CycleGAN-and-pix2pix 

In [ ]:
from sandbox import _package_dir
package_dir=os.path.abspath(_package_dir+"/../../pytorch-CycleGAN-and-pix2pix")

## Import the desired module

In [ ]:
from sandbox.modules.pytorch import LandscapeGeneration
landscape = LandscapeGeneration(extent=sensor.extent, package_dir=package_dir)

In [ ]:
import os
from sandbox import _test_data
os.listdir(_test_data["landscape_generation"] + "checkpoints")

## Add the module to the main thread

In [ ]:
main.add_module('landscape', landscape)

### Be sure that the model you want to use is saved in the checkpoint folder

In [ ]:
assert len(landscape.name_trained_models)>0
landscape.name_model = landscape.name_trained_models[0]

#Now you can select which model you want to select, by default it will use the model that ppears first in the list
print("Possible models: ", landscape.name_trained_models, "\n Model to use:", landscape.name_model)

### Use the widgets from the LoadSaveTopoModule to set the area for the landscape generation

Use the tab "Box widgets" to modify the extent of the box and capture a new frame using the snapshot button

In [ ]:

landscape.LoadArea.show_widgets()

It is also possible to this manually by:

In [ ]:
save_topo = landscape.LoadArea.extractTopo()
assert landscape.LoadArea.absolute_topo is not None

### Now with the saved topography, we first need to modify and save the image to be used by pytorch-CycleGAN-and-pix2pix

In [ ]:
DEM = landscape.get_image_modify()

In [ ]:
landscape.save_image(DEM, name = 'landscape_image.png')

### If the previous was succesfull and you can see the doubled image, you can continue by running the command line using the follow line: 

In [ ]:
landscape.run_cmd()#package_dir=package_dir)

This basically will construct the line you need to run in your terminal or cmd for pytorch-CycleGAN-and-pix2pix to run correctly in your system. If the line is not working, you can run this manually in your system. Just copy paste the previous

### When this is done, you can load and display the generated image by:

In [ ]:
landscape.read_result(name = 'landscape_image.png')

In [ ]:
import matplotlib.pyplot as plt
main.stop()
fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(landscape.DEM, origin="lower", cmap = "gist_earth")
ax[0].set_title("Real DEM Image")
ax[0].set_axis_off()

ax[1].imshow(landscape.img,  extent=landscape.LoadArea.to_box_extent)
ax[1].set_title("Generated Landscape Image")
ax[1].set_axis_off()
plt.show()
main.run()

### And if you want to visualize this image on the sandbox change this flag to True: 

In [ ]:
landscape.show_landscape=True










## All the previous work also with widgets following the instructions and previous procedure

In [ ]:
landscape.show_widgets()

In [ ]:












main.stop()